In [1]:
!pip install requests

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.0 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import requests
import json

import streamlit as st
import pandas as pd
import requests
import json

# Function to call the Azure OpenAI API
def chat_with_gpt(endpoint, api_key, prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    data = {
        "model": "gpt-4o",  # Specify the model
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 800,  # Adjust token limit as needed
    }

    response = requests.post(f"{endpoint}", headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code}, {response.text}"

# Streamlit user interface
st.title("ChatGPT-4o Azure Interface with File Upload")

# Input fields for endpoint and API key
endpoint = st.text_input("Azure OpenAI Endpoint", placeholder="https://<your-resource-name>.openai.azure.com/")
api_key = st.text_input("API Key", type="password")

# File uploader for multiple file types
uploaded_file = st.file_uploader("Upload a file (txt, csv)", type=["txt", "csv"])

# Text input for user prompt
user_prompt = st.text_area("Ask a question related to the uploaded file:")

# Button to submit the prompt
if st.button("Submit"):
    if endpoint and api_key and uploaded_file and user_prompt:
        # Read the uploaded file
        if uploaded_file.type == "text/plain":
            content = uploaded_file.read().decode("utf-8")
            file_type = "text"
        elif uploaded_file.type == "text/csv":
            content = pd.read_csv(uploaded_file).to_string()  # Convert DataFrame to string
            file_type = "csv"
        else:
            st.warning("Currently, only text and CSV files are processed.")
            content = ""

        if content:
            # Combine the content and prompt
            full_prompt = f"Here is the content of the {file_type} file:\n{content}\n\nUser's Question: {user_prompt}"
            response = chat_with_gpt(endpoint, api_key, full_prompt)
            st.write("**Response:**")
            st.write(response)
    else:
        st.warning("Please fill all fields and upload a file.")

Writing app.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

35.227.59.88


In [6]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.59.88:8501

your url is: https://spotty-grapes-tie.loca.lt
  Stopping...
^C
